
# 07 — RAG Evaluation & Benchmarks  
RAGAS-style evaluation • LLM-as-judge • Retrieval metrics (Recall@K, Precision@K)  
Fully runnable notebook.



## 1. Synthetic Evaluation Dataset Generator


In [ ]:

# Synthetic evaluation dataset
eval_data = [
    {
        "question": "What is machine learning?",
        "ground_truth": "Machine learning is a field focused on algorithms that learn from data."
    },
    {
        "question": "Define training data.",
        "ground_truth": "Training data is labeled data used by models to learn patterns."
    }
]

eval_data



## 2. Basic Retrieval Setup


In [ ]:

!pip install langchain langchain-openai chromadb langchain-community --quiet

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

# Sample corpus
with open("corpus.txt", "w") as f:
    f.write("Machine learning is a method for training models using data.\n")
    f.write("Training data consists of labeled examples that teach the model.\n")

loader = TextLoader("corpus.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.split_documents(docs)

emb = OpenAIEmbeddings()
db = Chroma.from_documents(chunks, emb)

db



## 3. Retrieval Metrics (Recall@K, Precision@K)


In [ ]:

from langchain_openai import OpenAIEmbeddings

def get_top_k(query, k=3):
    return db.similarity_search(query, k=k)

# Compute simple Recall@K
def recall_at_k(question, ground_truth, k=3):
    retrieved = get_top_k(question, k)
    hits = [ground_truth in d.page_content for d in retrieved]
    return sum(hits) / 1  # 1 relevant doc

# Evaluate dataset
for item in eval_data:
    r = recall_at_k(item["question"], item["ground_truth"], k=3)
    print(item["question"], "Recall@3 =", r)



## 4. RAGAS-like Faithfulness & Relevance Evaluation


In [ ]:

llm = ChatOpenAI(model="gpt-4o-mini")

def answer_rag(question):
    docs = get_top_k(question, k=3)
    ctx = "\n".join([d.page_content for d in docs])
    prompt = f"Context:\n{ctx}\n\nQuestion:{question}\nAnswer:"
    return llm.invoke(prompt).content

def faithfulness_score(answer, context):
    # LLM-as-judge scoring
    prompt = f"""Rate if answer is grounded in context (0-5).
Context:
{context}

Answer:
{answer}

Score only number."""
    score = llm.invoke(prompt).content.strip()
    try:
        return float(score)
    except:
        return 0.0

# Evaluate
for item in eval_data:
    ans = answer_rag(item["question"])
    ctx = "\n".join([d.page_content for d in get_top_k(item["question"], k=3)])
    score = faithfulness_score(ans, ctx)
    print(item["question"], "Faithfulness:", score)



## 5. LLM-as-Judge Full Evaluation Pipeline


In [ ]:

def judge(question, answer, ground_truth):
    prompt = f"""Evaluate answer compared to ground truth.
Return score 0-5.

Question: {question}
Answer: {answer}
Ground Truth: {ground_truth}
Score:
"""
    s = llm.invoke(prompt).content.strip()
    try:
        return float(s)
    except:
        return 0.0

for item in eval_data:
    ans = answer_rag(item["question"])
    score = judge(item["question"], ans, item["ground_truth"])
    print(item["question"], "LLM-as-judge score:", score)



## End of Notebook 07  
Fully generated.
